In [2]:
#link test for your api_key and base_url
from openai import OpenAI
API_KEY = "sk-9aJdh7rHjW4tbGWMvS360F3pE0FAd15e1Br80vc8mNuQLr6g"
BASE_URL = "https://api.deepbricks.ai/v1/"
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": """Hello!"""
        }
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Hey! How’s it', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, thought_content='')


In [1]:
from openai import OpenAI
import traceback  # 用于获取详细异常信息

def annotate_code(target: str) -> str:
    """
    调用大语言模型对代码字符串进行标注
    
    参数:
        target: 待标注的代码字符串
        
    返回:
        标注后的代码字符串，失败时返回空字符串
    """
    # 配置API客户端
    API_KEY = "sk-9aJdh7rHjW4tbGWMvS360F3pE0FAd15e1Br80vc8mNuQLr6g"
    BASE_URL = "https://api.deepbricks.ai/v1/"
    
    try:
        # 初始化客户端
        client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        
        # 构建标注提示词
        system_prompt = """
        你需要完成Python代码的标注任务。标注规则如下：
        1. 标注目标：对代码中进行了API调用的操作，包含格式转换，外部库调用，系统操作等进行标注。这些代码可能是不完整的，那么不必理会最后无法解析的部分代码，只需要对前面的代码标注即可，不完整的代码原样返回。
        2. 标注格式：在目标API调用语句前添加 '<LibFunc->(Query)>'标记，其中(Query)必须包含：
           - 操作主体（如tokenizer、generator、model、websocket、os等）
           - 操作行为（如encode文本、generate单个token、forward推理、拼接路径等）
        3. 标注示例：
           - 原始代码：output = tokenizer.encode(text) → 标注后：output = <LibFunc->(use tokenizer to encode the text)>tokenizer.encode(text)
           - 原始代码：new_token = generator.gen_single_token() → 标注后：new_token = <LibFunc->(use generator to generate a single token)>generator.gen_single_token()
        4. 输出要求：仅返回「标注后的代码片段」，不包含任何多余解释、注释或格式说明，不要改动任何代码，包含换行符，空格等结构信息。
        
        以下是几个合适的例子：
        1. "\n\n    output = <LibFunc->(decode and return the first text in the sequence)>tokenizer.decode(generator.sequence[0])\n    return output\n\nfor i in range(10):\n\n    alpha = i / 5.0 - 0.4\n    print()\n    print(f\"--------------------------------------\")\n    print(f\"alpha = {alpha:.1f}\")\n    print(f\"--------------------------------------\")\n    output = <LibFunc->(call generate_cfg to generate text no more than 200 tokens)>generate_cfg(prompts, alpha, 200)\n    <LibFunc->(print the result without prefix)>print(output[len(prompts[0]):].strip())\n"
        2. "\n    input_ids = input_ids[:, -max_input_tokens:]\n    prompt_ids = input_ids\n\n    full_prompt = prompt\n    utilized_prompt = <LibFunc->(use tokenizer to decode prompt_ids)>tokenizer.decode(prompt_ids)[0]\n    built_response = \"\"\n\n    remaining_tokens = max_new_tokens\n\n    # Settings\n\n    stop_strings = []\n    stop_tokens = []\n    for t in stop_conditions:\n        if isinstance(t, int): stop_tokens += [t]\n        if isinstance(t, str): stop_strings += [t]\n\n    held_text = \"\"\n\n    max_stop_string = 2\n    for ss in stop_strings:\n        max_stop_string = max(max_stop_string, get_num_tokens(ss) + 2)\n\n    generator.settings = gen_settings\n\n    # Start generation\n\n    <LibFunc->(use generator to generate)>generator.gen_begin_reuse(input_ids)\n\ndef stream():\n    global model, cache, config, generator, tokenizer\n    global stop_strings, stop_tokens, prompt_ids, held_text, max_stop_string, remaining_tokens\n    global full_prompt, utilized_prompt, built_response\n\n    # Check total response length\n\n    if remaining_tokens == 0:\n        return held_text, True, full_prompt + built_response, utilized_prompt + built_response, built_response\n    remaining_tokens -= 1\n\n    # Generate\n\n    old_tail = <LibFunc->(use tokenizer to decode...)>tokenizer.decode(generator."
        3. "import sys\n\ndef main():\n    T = <LibFunc->(read line from stdin and convert to integer)>int(sys.stdin.readline())\n    for _ in range(T):\n        R, C = <LibFunc->(read line from stdin, split it and convert to integers)>map(int, sys.stdin.readline().split())\n        grid = [<LibFunc->(read line from stdin and remove trailing whitespace)>sys.stdin.readline().strip() for _ in range(R)]\n        count = 0\n        for i in range(R):\n            for j in range(C):\n                if grid[i][j] == '#':\n                    continue\n                # Compute L\n                L = 0\n                current_j = j - 1\n                while current_j >= 0 and grid[i][current_j] == '^':\n                    L += 1\n                    current_j -= 1\n                # Compute R\n                R_count = 0\n                current_j = j + 1\n                while current_j < C and grid[i][current_j] == '^':\n                    R_count += 1\n                    current_j += 1\n                # Compute T\n                T = 0\n                current_i = i - 1\n                while current_i >= 0 and grid[current_i][j] == '^':\n                    T += 1\n                    current_i -= 1\n                # Compute B\n                B = 0\n                current_i = i + 1\n                while current_i < R and grid[current_i][j] == '^':\n                    B += 1\n                    current_i += 1\n                min_val = <LibFunc->(get minimum of four values)>min(L, R_count, T, B)\n                if min_val >= 2:\n                    count += 1\n        <LibFunc->(print the final count)>print(count)\n\nif __name__ == \"__main__\":\n    main()"
        """
        
        # 调用大语言模型
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"请标注以下代码：\n\'{target}\'"}
            ],
            temperature=0.1,
            timeout=1000
        )
        
        # 提取并返回标注结果
        return response.choices[0].message.content.strip() if response.choices else ""
        
    except Exception as e:
        # 打印包含具体原因的异常信息
        print(f"Exception occurred: {str(e)}")
        # 可选：如果需要更详细的堆栈信息，可取消下面这行注释
        # print(f"详细错误信息:\n{traceback.format_exc()}")
        return ""

In [2]:
import os
import json
from tqdm import tqdm
def annotate(n: int) -> None:
    """
    处理数据集的前n个对象，对所有prefix_split和suffix_split字段调用annotate_code，
    使用tqdm显示每个对象内的分割字段处理进度，并将结果保存到新文件
    
    参数:
        n: 要处理的数据对象数量
    """
    # 文件路径配置
    input_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split.json"
    output_path = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split_annotated.json"
    
    # 检查输入文件是否存在
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"输入文件未找到：{input_path}")
    
    # 读取分割后的数据集
    with open(input_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    # 验证n的有效性
    if n <= 0 or n > len(dataset):
        raise ValueError(f"n的值必须在1到{len(dataset)}之间（当前数据集大小）")
    
    # 处理前n个对象（外层进度条：对象级）
    processed_dataset = []
    for obj_idx, item in enumerate(tqdm(dataset[:n], desc="总进度", position=0)):
        # 复制原始对象（包含completion字段）
        annotated_item = {"completion": item["completion"]}
        
        # 收集所有需要处理的分割字段
        all_split_fields = [
            key for key in item 
            if key.startswith("prefix_split_") or key.startswith("suffix_split_")
        ]
        
        # 处理当前对象内的所有分割字段（内层进度条：字段级）
        for field in tqdm(
            all_split_fields, 
            desc=f"对象 {obj_idx+1}/{n} 处理中", 
            position=1, 
            leave=False  # 处理完一个对象后清除内层进度条
        ):
            # 调用annotate_code进行标注
            temp = annotate_code(item[field])
            if temp == "":
                temp = annotate_code(item[field])
            annotated_content = temp
            annotated_item[field] = annotated_content
        
        processed_dataset.append(annotated_item)
    
    # 保存标注后的数据集
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 标注完成！已处理前{n}个对象，结果保存至：")
    print(f"   {output_path}")
    
    # 验证标注结果
    if processed_dataset:
        first_item = processed_dataset[0]
        sample_prefix = next(key for key in first_item if key.startswith("prefix_split_"))
        sample_suffix = next(key for key in first_item if key.startswith("suffix_split_"))
        
        print("\n标注结果示例（前20个字符）：")
        print(f"{sample_prefix}：{first_item[sample_prefix][:20]}...")
        print(f"{sample_suffix}：{first_item[sample_suffix][:20]}...")

In [4]:
annotate(50)

总进度: 100%|██████████| 50/50 [51:35<00:00, 61.90s/it]            


✅ 标注完成！已处理前50个对象，结果保存至：
   C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split_annotated.json

标注结果示例（前20个字符）：
prefix_split_1：```python
import tki...
suffix_split_1：```
        <LibFunc...


In [19]:
# tiny cell for repairing missing code

fix_string = "\n\n    return quadform\n\n\ndef logdet_product(*args, **kwargs):\n    r\"\"\"Compute the log-determinant of a product of matrices.\n\n    Here, \"product\" refers to $X = A^\\top A$.\n    \"\"\"\n    return trace_of_matfun_product(np.log, *args, **kwargs)\n\n\ndef schatten_norm(*args, power, **kwargs):\n    r\"\"\"Compute the Schatten-p norm of a matrix via stochastic Lanczos quadrature.\"\"\"\n\n    def matfun(x):\n        \"\"\"Matrix-function for Schatten-p norms.\"\"\"\n        return x ** (power / 2)\n\n    trace = trace_of_matfun_product(matfun, *args, **kwargs)\n    return trace ** (1 / power)\n\n\ndef trace_of_matfun_product(matfun, order, *matvec_funs, matrix_shape, **kwargs):\n    r\"\"\"Compute the trace of a function of a product of matrices.\n\n    Here, \"product\" refers to $X = A^\\top A$.\n    \"\"\"\n    quadratic_form = _quadratic_form_slq_product(\n        matfun, order, *matvec_funs, matrix_shape=matrix_shape\n    )\n    return montecarlo.estimate(quadratic_form, **kwargs)\n\n\ndef _quadratic_form_slq_product(matfun, depth, *matvec_funs, matrix_shape):"
result = annotate_code(fix_string)
if result == "":
    result = annotate_code(fix_string)
print(repr(result))

'```\n    return quadform\n\n\ndef logdet_product(*args, **kwargs):\n    r"""Compute the log-determinant of a product of matrices.\n\n    Here, "product" refers to $X = A^\\top A$.\n    """\n    return <LibFunc->(use trace_of_matfun_product with numpy.log to compute log-determinant)>trace_of_matfun_product(np.log, *args, **kwargs)\n\n\ndef schatten_norm(*args, power, **kwargs):\n    r"""Compute the Schatten-p norm of a matrix via stochastic Lanczos quadrature."""\n\n    def matfun(x):\n        """Matrix-function for Schatten-p norms."""\n        return x ** (power / 2)\n\n    trace = <LibFunc->(use trace_of_matfun_product with matfun to compute trace)>trace_of_matfun_product(matfun, *args, **kwargs)\n    return trace ** (1 / power)\n\n\ndef trace_of_matfun_product(matfun, order, *matvec_funs, matrix_shape, **kwargs):\n    r"""Compute the trace of a function of a product of matrices.\n\n    Here, "product" refers to $X = A^\\top A$.\n    """\n    quadratic_form = <LibFunc->(call _quadra